In [595]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
pd.set_option('display.max_columns', None)


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import plotly.express as px

In [596]:
raw_data = pd.read_csv('Data/data_clean')

In [597]:
raw_data['Ausführende Firma'].value_counts()

Gegenbauer       198
Lindner          137
B&M               88
KOHA              47
Spektral          25
Hildebrandt       18
Vieweg            10
Behörde           10
MBM                5
Vidacek            3
Jahnke             2
FolienKartell      2
RNC                1
Allianz            1
Folientechnik      1
Name: Ausführende Firma, dtype: int64

In [598]:
# remove companies that have less than 2 invoices
list_of_contractors = ['B&M', 'Gegenbauer', 'Behörde',
       'Spektral', 'Hildebrandt', 'Jahnke', 'KOHA',
       'Lindner', 'MBM', 'Vieweg', 'Vidacek']

raw_data = raw_data[raw_data['Ausführende Firma'].isin(list_of_contractors)].iloc[:, 1:]

In [599]:
# create a column for contractor_type
keys_list = list_of_contractors

values_list = ['Painter/Plasterer','Cleaning company','Permit office','Scaffold-builder','Cleaning company',
               'Roofer', 'Shell/Core company', 'Interior outfitter', 'Facade/Glass builder',
              'Electrician', 'Surveyor']

zip_iterator = zip(keys_list, values_list)

contractor_type = dict(zip_iterator)
    

In [600]:
raw_data['contractor_type'] = raw_data['Ausführende Firma'].map(contractor_type)

In [601]:
raw_data.head()

,Datum,reference_nr,Ausführende Firma,Sachzusammenhang,Leistungs-beschreibung,Bauteil,GP[€],Weiterbelastung an Firma,Weiterbelastungs-betrag [€],Gründe,Nachweis_art,Stundenarbeit,contractor_type
1,2019-04-18,2298,B&M,Beschädigung der Attika,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Jahnke,121.8,Mangelbehebung,Stundenzettel,21.0,Painter/Plasterer
2,2019-04-18,2298,B&M,Beschädigung der Attika,Attika: Nachgedämmte Flächen mit Armierung ein...,BE,1096.2,MBM,121.8,Mangelbehebung,Stundenzettel,21.0,Painter/Plasterer
3,2019-04-18,2298,B&M,Beschädigung der Attika,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Hauke Runge,121.8,Mangelbehebung,Stundenzettel,21.0,Painter/Plasterer
4,2019-04-18,2298,B&M,Beschädigung der Attika,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Koha,121.8,Mangelbehebung,Stundenzettel,21.0,Painter/Plasterer
5,2019-04-18,2298,B&M,Beschädigung der Attika,Attika: Nachgedämmte Flächen mit Armierung ein...,BG1,1096.2,Vieweg,121.8,Mangelbehebung,Stundenzettel,21.0,Painter/Plasterer


In [602]:
raw_data.columns

Index(['Datum', 'reference_nr', 'Ausführende Firma', 'Sachzusammenhang',
       'Leistungs-beschreibung', 'Bauteil', 'GP[€]',
       'Weiterbelastung an Firma', 'Weiterbelastungs-betrag [€]', 'Gründe',
       'Nachweis_art', 'Stundenarbeit', 'contractor_type'],
      dtype='object')

In [603]:
data_renamed = raw_data[['Datum', 'reference_nr',
       'Leistungs-beschreibung', 'Bauteil', 'GP[€]',
       'Weiterbelastung an Firma', 'Weiterbelastungs-betrag [€]',
       'Nachweis_art', 'Stundenarbeit', 'contractor_type']]



data_renamed.columns = ['Date', 'invoice_Nr.', 'work_description', 'building', 'total_cost',
                        'reimbursement_rqst', 'reimbursement_amount', 'proof_type',
                        'total_manhours', 'contractor_type']

In [604]:
data_renamed.head()

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
1,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Jahnke,121.8,Stundenzettel,21.0,Painter/Plasterer
2,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BE,1096.2,MBM,121.8,Stundenzettel,21.0,Painter/Plasterer
3,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Hauke Runge,121.8,Stundenzettel,21.0,Painter/Plasterer
4,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Koha,121.8,Stundenzettel,21.0,Painter/Plasterer
5,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BG1,1096.2,Vieweg,121.8,Stundenzettel,21.0,Painter/Plasterer


In [605]:
data_renamed['reimbursement_rqst'].unique()

array(['Jahnke', 'MBM', 'Hauke Runge', 'Koha', 'Vieweg', 'MFT', 'Lindner',
       'EN+', 'B&M', 'Suhrkamp', 'Bauwesensversicherung', 'in Prüfung',
       'keine\n Weiterbelastung (BH)', 'Protec', 'Vattenfall',
       'Mieterwunsch Creandum', 'BA', 'En+', 'Mieterwunsch/ BH',
       'Bundschuh', 'hhvh'], dtype=object)

In [606]:
keys_list = data_renamed['reimbursement_rqst'].unique()

values_list = ['Roofer', 'Facade/Glass builder', 'Water/Plumbing fitter',
 'Shell/Core company', 'Electrician', 'Metal fabricator',
 'Interior outfitter','TBE', 'Painter/Plasterer','Tenant','Insurance',
 'Not_clear','Building-owner', 'TBE', 'Water provider', 'Tenant', 'other','TBE','Building-owner',
               'Architect', 'Landscaper']

zip_iterator = zip(keys_list, values_list)
my_dict = dict(zip_iterator)
data_renamed['reimbursement_rqst'] = data_renamed['reimbursement_rqst'].map(my_dict)

In [607]:
data_renamed.head()

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
1,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Roofer,121.8,Stundenzettel,21.0,Painter/Plasterer
2,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BE,1096.2,Facade/Glass builder,121.8,Stundenzettel,21.0,Painter/Plasterer
3,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Water/Plumbing fitter,121.8,Stundenzettel,21.0,Painter/Plasterer
4,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Shell/Core company,121.8,Stundenzettel,21.0,Painter/Plasterer
5,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BG1,1096.2,Electrician,121.8,Stundenzettel,21.0,Painter/Plasterer


In [608]:
data_renamed['proof_type'].unique()

array(['Stundenzettel', 'nachtrag', 'Sondernutzung', 'rechnung', nan,
       'mehrkosten', 'Auftrag'], dtype=object)

In [609]:
key_list = data_renamed['proof_type'].unique()

value_list = ['timesheet', 'additional work agreement',
          'permit', 'invoice','other', 'additional cost agreement',
         'contract']

data_renamed['proof_type'] = data_renamed['proof_type'].map(dict(zip(key_list, value_list)))

In [610]:
data_renamed.head()

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
1,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Roofer,121.8,timesheet,21.0,Painter/Plasterer
2,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BE,1096.2,Facade/Glass builder,121.8,timesheet,21.0,Painter/Plasterer
3,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Water/Plumbing fitter,121.8,timesheet,21.0,Painter/Plasterer
4,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Shell/Core company,121.8,timesheet,21.0,Painter/Plasterer
5,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,BG1,1096.2,Electrician,121.8,timesheet,21.0,Painter/Plasterer


In [611]:
data_renamed['building'].unique()

array([nan, 'BE', 'BG1', 'BG2', 'WG', 'wird geprüft', 'allg.', 'Alle'],
      dtype=object)

In [612]:
my_dict = {'BG1' :'building1', 'BG2': 'building2', 'WG': 'building3',
        'wird geprüft':'BE', 'allg.': 'BE', 'Alle': 'all'}
data_renamed['building'] = data_renamed['building'].map(my_dict)

In [613]:
data_renamed.head()

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
1,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Roofer,121.8,timesheet,21.0,Painter/Plasterer
2,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Facade/Glass builder,121.8,timesheet,21.0,Painter/Plasterer
3,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Water/Plumbing fitter,121.8,timesheet,21.0,Painter/Plasterer
4,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Shell/Core company,121.8,timesheet,21.0,Painter/Plasterer
5,2019-04-18,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,building1,1096.2,Electrician,121.8,timesheet,21.0,Painter/Plasterer


In [614]:
data_renamed.describe()

,invoice_Nr.,total_cost,reimbursement_amount,total_manhours
count,543.000000,538.000000,528.000000,433.000000
mean,5393.513812,1508.648926,439.104943,15.702656
std,2608.508146,3969.380083,1162.958408,26.981327
min,1005.000000,0.000000,0.000000,0.500000
25%,2725.000000,238.669500,31.780000,4.500000
50%,5569.000000,457.600000,76.270000,11.500000
75%,7761.000000,1096.200000,369.000000,22.000000
max,9905.000000,31871.558000,11358.120000,377.000000


In [615]:
data_renamed[data_renamed['contractor_type'] == 'Facade/Glass builder']

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
452,2019-01-25,4023,Ertüchtigung Bransperre,building1,24465.24,Building-owner,0.0,additional work agreement,NaN,Facade/Glass builder
453,2019-04-26,2466,Entfernung Dachpappe für Montage PR Fassade EG,building1,159.00,Building-owner,0.0,timesheet,3.0,Facade/Glass builder
454,2019-07-01,2622,Einbau/ Wiedermontage Bautür inkl. Holzverschlag,building1,424.00,Building-owner,0.0,timesheet,8.0,Facade/Glass builder
455,2019-06-28,6948,Demontage Bauteile inkl. Holzverschlag für Ein...,building1,212.00,Building-owner,0.0,timesheet,4.0,Facade/Glass builder
456,2019-01-16,7075,Beräumung EG Fläche zur Montage PF Fassade,building1,927.50,Building-owner,0.0,timesheet,17.5,Facade/Glass builder


In [616]:
from datetime import timedelta
from dateutil.relativedelta import relativedelta

data_renamed['Date'] = pd.to_datetime(data_renamed['Date'])

In [617]:
# move dates by 3 years and keep only month
data_renamed['Date'] = data_renamed['Date'] - pd.offsets.DateOffset(years=2)

In [618]:
data_renamed[data_renamed['Date'] >'2019-12-12']#

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
499,2205-03-11,3482,Montage einer Vorsatzschale in der in der Unte...,building1,396.0,Building-owner,NaN,timesheet,8.0,Interior outfitter
500,2205-03-11,3482,Montage einer Vorsatzschale in der in der Unte...,building2,396.0,Building-owner,NaN,timesheet,8.0,Interior outfitter


In [619]:
data_renamed.at[499, 'Date'] = '2017-03-11'
data_renamed.at[500, 'Date'] = '2017-03-11'

In [620]:
data_renamed

,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
1,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Roofer,121.8,timesheet,21.0,Painter/Plasterer
2,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Facade/Glass builder,121.8,timesheet,21.0,Painter/Plasterer
3,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Water/Plumbing fitter,121.8,timesheet,21.0,Painter/Plasterer
4,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Shell/Core company,121.8,timesheet,21.0,Painter/Plasterer
5,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,building1,1096.2,Electrician,121.8,timesheet,21.0,Painter/Plasterer
...,...,...,...,...,...,...,...,...,...,...
545,NaT,9651,Verlängerung der Vorhaltung zu Pos. NA06-005:\...,building1,294.8,Facade/Glass builder,294.8,invoice,NaN,Scaffold-builder
546,NaT,5233,Verlängerung der Vorhaltung zu Pos. NA06-001\n...,building1,168.0,Facade/Glass builder,168.0,invoice,NaN,Scaffold-builder
547,2017-06-03 00:00:00,4810,"gem. PTC: Aufspitzen Fussboden, 2 h",building3,70.0,Water/Plumbing fitter,70.0,timesheet,2.0,Electrician
548,NaT,4673,Verlängerung der Vorhaltung zu Pos.NA02-001: S...,building1,27.0,Facade/Glass builder,27.0,invoice,NaN,Scaffold-builder


In [621]:
data_renamed.to_csv('Data/df_cleaned')

In [622]:
data_renamed = pd.read_csv('Data/df_cleaned')
data_renamed = data_renamed.iloc[:,:]
data_renamed

,Unnamed: 0,Date,invoice_Nr.,work_description,building,total_cost,reimbursement_rqst,reimbursement_amount,proof_type,total_manhours,contractor_type
0,1,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Roofer,121.8,timesheet,21.0,Painter/Plasterer
1,2,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Facade/Glass builder,121.8,timesheet,21.0,Painter/Plasterer
2,3,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Water/Plumbing fitter,121.8,timesheet,21.0,Painter/Plasterer
3,4,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,NaN,1096.2,Shell/Core company,121.8,timesheet,21.0,Painter/Plasterer
4,5,2017-04-18 00:00:00,2298,Attika: Nachgedämmte Flächen mit Armierung ein...,building1,1096.2,Electrician,121.8,timesheet,21.0,Painter/Plasterer
...,...,...,...,...,...,...,...,...,...,...,...
538,545,NaN,9651,Verlängerung der Vorhaltung zu Pos. NA06-005:\...,building1,294.8,Facade/Glass builder,294.8,invoice,NaN,Scaffold-builder
539,546,NaN,5233,Verlängerung der Vorhaltung zu Pos. NA06-001\n...,building1,168.0,Facade/Glass builder,168.0,invoice,NaN,Scaffold-builder
540,547,2017-06-03 00:00:00,4810,"gem. PTC: Aufspitzen Fussboden, 2 h",building3,70.0,Water/Plumbing fitter,70.0,timesheet,2.0,Electrician
541,548,NaN,4673,Verlängerung der Vorhaltung zu Pos.NA02-001: S...,building1,27.0,Facade/Glass builder,27.0,invoice,NaN,Scaffold-builder
